In [10]:
admin_names = ["Dev Coder", "MM Coder","Exchange Coder"]
account_names = ["Thanh Bao Coder", "Tran Ngoc Chinh Coder", "Sai Coder","Nguyen Coder","Nhi Coder"]
DATA = [
    [
       {"from": "", "to": "Dev Coder", "amount": 30000},
       {"from": "", "to": "MM Coder", "amount": 30000},
       {"from": "", "to": "Exchange Coder", "amount": 30000},
    ],
    [
        {"from": "", "to": "Thanh Bao Coder", "amount": 100},
        {"from": "Thanh Bao Coder", "to": "Tran Ngoc Chinh Coder", "amount": 10},
        {"from": "Tran Ngoc Chinh Coder", "to": "Sai Coder", "amount": 5}
    ],
    [
        {"from": "Tran Ngoc Chinh Coder", "to": "Thanh Bao Coder", "amount": 3},
        {"from": "MM Coder", "to": "Thanh Bao Coder", "amount": 50},
        {"from": "Thanh Bao Coder", "to": "Dev Coder", "amount": 5}
    ]
]
import hashlib, json, random
from datetime import datetime, timedelta

# tạo một block
class Block:
    def __init__(self,data) :
        self.data = data
        self.prev_hash = ""
        self.nonce = 0
        self.hash = ""
        self.total_time = ""

# tạo hàm băm data với thuật toán sha512
def hash(block):
    data = json.dumps(block.data) + block.prev_hash + str(block.nonce)
    data = data.encode('utf-8')
    return hashlib.sha256(data).hexdigest()
# tạo blockchain la chuoi cac block
class Blockchain:
    def __init__(self,miner) :
        self.miner = miner
        self.chain = [] # genesis block
        self.reward = 50  # Phần thưởng ban đầu
        self.halving_interval = 3000  # Số block giảm phần thưởng


        genesis_block = Block("Geneis Block")
        genesis_block.hash = hash(genesis_block)

        self.chain.append(genesis_block)

    def get_block_subsidy(self, height):
        halvings = height // self.halving_interval
        # Force block reward to zero when right shifting is undefined.
        if halvings >= 64:
            return 0

        # Subsidy is cut in half every halving interval
        n_subsidy = self.reward // (2 ** halvings)
        return n_subsidy

    def add_block(self, data):
         # Kiểm tra giới hạn số lượng token trong một giao dịch
        for transaction in data:
            if transaction["amount"] > 999999:
                print("Transaction amount exceeds the limit (999.999). Block not added.")
                return

        total_transactions = len(data)
        block=Block(data)

        block.data.append({"from":"", "to": self.miner, "amount": self.get_block_subsidy(len(self.chain) - 1) + total_transactions}) # thưởng cho miner khi mã hóa được block

        block.prev_hash = self.chain[-1].hash
        block.hash = hash(block)
        start = datetime.now()  # bắt đầu đếm time
        elapsed_time = timedelta()
        while not hash(block).startswith("0") or elapsed_time < timedelta(seconds=0):
            block.nonce += 1
            block.hash = hash(block)
            elapsed_time = datetime.now() - start
        end = datetime.now()  # ket thuc đếm time
        block.total_time = str(end-start)

        # Kiểm tra số dư của mỗi tài khoản
        if not self.is_balance_valid():
            return
        #  Kiểm tra tổng số token của tất cả các tài khoản và miner
        elif not self.sum_balance_valid():
            return
        else:
            self.chain.append(block)
     


    def print(self):
      chain_number = 1

      # duyet cac phan tu trong list
      for block in self.chain:
          print('Block number:', chain_number)
          print("Data:", block.data)
          print("Previous hash:", block.prev_hash)
          print("Hash:", block.hash)
          print("Nonce:", block.nonce)
          print("Total time:", block.total_time)
          print("-------------------------------------------------------------------------------------------------------")
          print("")
          chain_number += 1
    def is_valid(self):
        for i in range(1, len(self.chain)):

            current_block = self.chain[i]
            prev_block =  self.chain[i-1]

            if hash(current_block) != current_block.hash:
                return False
            if prev_block.hash != current_block.prev_hash:
                return False
            # check thêm điều kiện
        return True
    def sum_balance_valid(self):
        sum=0
        accounts = set()
        for block in self.chain:
            if isinstance(block.data, list):
                for transaction in block.data:
                    accounts.add(transaction["from"])
                    accounts.add(transaction["to"])
                accounts.discard("")
        for account in accounts:
            balance = self.get_balance(account)
            sum = sum + balance
        if sum >1000000:
                print("Sum of all tokens exceeds the limit (1.000.000). Block not added.")
                self.chain.pop()
                return False
        return True

    def is_balance_valid(self):
        accounts = set()
        for block in self.chain:
            if isinstance(block.data, list):
                for transaction in block.data:
                    accounts.add(transaction["from"])
                    accounts.add(transaction["to"])
                accounts.discard("")
        for account in accounts:

            balance = self.get_balance(account)
            # Kiểm tra số dư của mỗi tài khoản
            if not (0 <= balance <= 1000000):
                print(f"Balance of {account} ({balance} CTC) is not within the valid range (0, 1.000.000). Block not added.")
                self.chain.pop()
                return False
        return True

    def get_balance(self,person):
        balance = 0  # số dư
        for block in self.chain:
            if type(block.data) != list :
                continue
            for transfer in block.data :
                if transfer["from"] == person:   # nếu chuyển đi thì bị trừ tiền
                    balance = balance -transfer["amount"] - random.uniform(0.1, 1)
                if transfer["to"] == person:   # nếu nhận vào thì + tiền
                    balance = balance +transfer["amount"]
        return balance
    def print_all_accounts(self):
        accounts = set()
        total_token = 0
        for block in self.chain:
            if isinstance(block.data, list):
                for transaction in block.data:
                    accounts.add(transaction["from"])
                    accounts.add(transaction["to"])
                accounts.discard("")
        for account in accounts:
            balance = self.get_balance(account)
            total_token = total_token + balance
        account_balances = {account: self.get_balance(account) for account in accounts}
        sorted_account_balances = sorted(account_balances.items(), key=lambda item: item[1], reverse=True)
        print("All Accounts:")
        print(f"Total Token: {total_token} CTC")
        for account, balance in sorted_account_balances:
            print(f"  {account}: {balance} CTC")
        print("-------------------------------------------------------------------------------------------------------")
    def get_accounts(self):
      accounts = set()
      for block in self.chain:
          if isinstance(block.data, list):
              for transaction in block.data:
                  accounts.add(transaction["from"])
                  accounts.add(transaction["to"])
              accounts.discard("")
      return accounts
    def stake_in_pool(self, pool_number,account_name,stake_amount):
      if pool_number not in {1, 2, 3}:
          print("Invalid pool number.")
          return
      locked_period = 0
      staking_fee = 0

      if pool_number == 1:
          locked_period = 3  # tháng
          staking_fee = 5
      elif pool_number == 2:
          locked_period = 6  # tháng
          staking_fee = 10
      elif pool_number == 3:
          locked_period = 12  # tháng
          staking_fee = 15
      # Kiểm tra số dư của tài khoản trước khi thêm stake
      if not (0 <= stake_amount <= 1000000):
          print("Stake amount is not within the valid range (0, 1.000.000). Stake not added.")
          return

      # Kiểm tra số dư của tài khoản
      if not self.is_balance_valid():
          return

      # Kiểm tra tổng số token của tất cả các tài khoản và miner
      elif not self.sum_balance_valid():
          return
      else:
          # Thêm giao dịch stake vào block
          stake_transaction = {"from": account_name, "to": f"Pool {pool_number}", "amount": stake_amount}
          fee_transaction = {"from": account_name, "to": "", "amount": staking_fee}
          self.add_block([stake_transaction, fee_transaction])

          # Xử lý lãi và phí staking
          locked_time = datetime.now() + timedelta(seconds=locked_period)
          print(f"Stake successful in Pool {pool_number}. Locked until {locked_time.strftime('%Y-%m-%d %H:%M:%S')}.")
          print("Interest will be calculated upon maturity.")
          print(f"Staking fee: {staking_fee} CTC deducted from your account.")
          print("-------------------------------------------------------------------------------------------------------")
          return locked_time

    def withdraw_stake(self, pool_number, locked_time,account_name,stake_amount):
        if pool_number not in {1, 2, 3}:
            print("Invalid pool number.")
            return
        withdraw_fee = 1
        current_time = datetime.now()
        if isinstance(locked_time, str):
          locked_time = datetime.strptime(locked_time, '%Y-%m-%d %H:%M:%S')
        if current_time < locked_time:
            print("Stake is still locked. Cannot withdraw before the lock period.")
        else:
            # Tính toán lãi và trả về tài khoản
            interest_earned = self.calculate_interest(locked_time, current_time, pool_number,stake_amount)
            withdrawal_amount = stake_amount
            # Thêm giao dịch rút stake và lãi vào block
            withdrawal_transaction = {"from": f"Pool {pool_number}", "to": account_name, "amount": withdrawal_amount-withdraw_fee}
            interest_transaction = {"from": "", "to": account_name, "amount": interest_earned-withdraw_fee}
            self.add_block([withdrawal_transaction, interest_transaction])

            print(f"Withdrawal successful from Pool {pool_number}.")
            print(f"Stake amount: {withdrawal_amount} CTC")
            print(f"Interest earned: {interest_earned} CTC")
            print("-------------------------------------------------------------------------------------------------------")

    def calculate_interest(self, start_time, end_time, pool_number,stake_amount):
        # Tính toán lãi dựa trên thời gian và lãi suất của pool
        time_diff = end_time - start_time
        locked_period = 0
        interest_rate = 0

        if pool_number == 1:
            locked_period = 3  # tháng
            interest_rate = 0.03  # 3%
        elif pool_number == 2:
            locked_period = 6  # tháng
            interest_rate = 0.08  # 8%
        elif pool_number == 3:
            locked_period = 12  # tháng
            interest_rate = 0.2  # 20%

         # Chuyển đổi chênh lệch thời gian sang đơn vị giây
        seconds_diff = time_diff.total_seconds()
        if seconds_diff - locked_period < 0:
            return 0  # Không tìm thấy thông tin giao dịch stake
        interest_earned = stake_amount * interest_rate
        return interest_earned
    def choose_pool(self):
        print("Choose a staking pool:")
        print("1. Pool 1 - Limit: 50000 tokens, Lock period: 3 months, Interest: 3%, Staking fee: 5")
        print("2. Pool 2 - Limit: 10000 tokens, Lock period: 6 months, Interest: 8%, Staking fee: 10")
        print("3. Pool 3 - Limit: 4000 tokens, Lock period: 12 months, Interest: 20%, Staking fee: 15")
        staking_fee = 0

        while True:
            try:
                pool_number = int(input("Enter the pool number (1, 2, or 3): "))
                account_name = input("Enter account name for staking: ")
                stake_amount = float(input("Enter the stake amount: "))
                
                # Set the maximum limit for each pool
                max_limit = {1: 50000, 2: 10000, 3: 4000}
                
                if pool_number in {1, 2, 3} and stake_amount <= max_limit[pool_number]:
                    if pool_number == 1:
                        staking_fee = 5
                    elif pool_number == 2:
                        staking_fee = 10
                    elif pool_number == 3:
                        staking_fee = 15

                    stake_amount = stake_amount - staking_fee
                    return pool_number, account_name, stake_amount
                else:
                    print(f"Invalid stake amount. Please enter a value less than or equal to {max_limit[pool_number]}.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")
    def has_staked(self, account_name, pool_number):
      for block in self.chain:
          if isinstance(block.data, list):
              for transaction in block.data:
                  if transaction["from"] == account_name and transaction["to"] == f"Pool {pool_number}":
                      return True
      return False

    def get_stake_info(self, account_name, pool_number):
        for block in reversed(self.chain):
            if isinstance(block.data, list):
                for transaction in block.data:
                    if transaction["from"] == account_name and transaction["to"] == f"Pool {pool_number}":
                        return  transaction["amount"]
        return 0

blockchain = Blockchain("Miner")

for _ in range(20):
      # Tạo số lượng giao dịch ngẫu nhiên từ 2 đến 4
      num_transactions = random.randint(2, 4)

      # Tạo giao dịch ngẫu nhiên
      new_transactions = []
      for _ in range(num_transactions):
          sender = random.choice(admin_names)  # Người gửi ngẫu nhiên
          receiver = random.choice(account_names)  # Người nhận ngẫu nhiên, có thể trùng với người gửi
          amount = random.uniform(0.1, 0.3)  # Số lượng token ngẫu nhiên

          transaction = {"from": sender, "to": receiver, "amount": amount}
          new_transactions.append(transaction)

      # Thêm block mới vào DATA
      DATA.append(new_transactions)
for block_transactions in DATA:
      if blockchain.is_balance_valid()==False:
          break
      else:
          blockchain.add_block(block_transactions)
      

blockchain.print_all_accounts()
  

All Accounts:
Total Token: 91278.89154243196 CTC
  Dev Coder: 29989.297348464253 CTC
  Exchange Coder: 29986.30857128188 CTC
  MM Coder: 29934.357809738303 CTC
  Miner: 1216 CTC
  Thanh Bao Coder: 138.5584440855301 CTC
  Sai Coder: 8.215006618382382 CTC
  Tran Ngoc Chinh Coder: 2.9138997282802555 CTC
  Nguyen Coder: 2.7093851065320624 CTC
  Nhi Coder: 1.9250243846933501 CTC
-------------------------------------------------------------------------------------------------------


In [11]:
def main():
  blockchain.print_all_accounts()
  # print("Is valid :",blockchain.is_valid())
  print("----------------------------------------- STAKE TOKEN -------------------------------------------------")
  STAKE_INFO = []
  while True:
      print("\nOptions:")
      print("1. Stake")
      print("2. Withdraw")
      print("3. Quit")

      choice = input("Enter your choice (1, 2, or 3): ")
      if choice == "1":
            chosen_pool, account_name, stake_amount = blockchain.choose_pool()
            locked_time = blockchain.stake_in_pool(chosen_pool, account_name, stake_amount)
            if locked_time is not None:
                # Save stake information
                stake_info = {"account_name": account_name, "pool_number": chosen_pool, "locked_time": locked_time, "stake_amount": stake_amount}
                STAKE_INFO.append(stake_info)
                blockchain.print_all_accounts()
      elif choice == "2":
            account_name = input("Enter account name for withdrawing: ")
            pool_number = int(input("Enter the pool number (1, 2, or 3): "))
            
            stake_info = next((info for info in STAKE_INFO if info["account_name"] == account_name and info["pool_number"] == pool_number), None)
            # Check if the account has staked in the chosen pool
            if blockchain.has_staked(account_name, pool_number):
                    locked_time = stake_info["locked_time"]
                    stake_amount = stake_info["stake_amount"]
                    blockchain.withdraw_stake(pool_number, locked_time, account_name, stake_amount)
                    blockchain.print_all_accounts()
                    # Remove stake information after withdrawal
                    STAKE_INFO.remove(stake_info)
            else:
                print(f"{account_name} has not staked in Pool {pool_number}. Cannot withdraw.")
      elif choice == "3":
            print("Exiting program.")
            break
      else:
            print("Invalid choice. Please enter 1, 2, or 3.")
if __name__ == "__main__":
    main()


All Accounts:
Total Token: 91281.50896804314 CTC
  Dev Coder: 29989.361805802277 CTC
  Exchange Coder: 29985.59048889975 CTC
  MM Coder: 29935.41859923232 CTC
  Miner: 1216 CTC
  Thanh Bao Coder: 137.37746009530034 CTC
  Sai Coder: 8.215006618382382 CTC
  Tran Ngoc Chinh Coder: 3.187896339112694 CTC
  Nguyen Coder: 2.7093851065320624 CTC
  Nhi Coder: 1.9250243846933501 CTC
-------------------------------------------------------------------------------------------------------
----------------------------------------- STAKE TOKEN -------------------------------------------------

Options:
1. Stake
2. Withdraw
3. Quit
Choose a staking pool:
1. Pool 1 - Limit: 50000 tokens, Lock period: 3 months, Interest: 3%, Staking fee: 5
2. Pool 2 - Limit: 10000 tokens, Lock period: 6 months, Interest: 8%, Staking fee: 10
3. Pool 3 - Limit: 4000 tokens, Lock period: 12 months, Interest: 20%, Staking fee: 15
Stake successful in Pool 1. Locked until 2023-12-01 10:49:50.
Interest will be calculated upon m